# Predicción del rendimiento académico usando árboles de decisión


Este cuaderno muestra cómo predecir el rendimiento académico de estudiantes a partir de sus hábitos.
Usaremos un modelo de árbol de decisión y aplicaremos validación cruzada (K-Fold) para mejorar la fiabilidad
de los resultados, dado que el dataset es pequeño.


In [1]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score


## Cargar el dataset

In [2]:

# Cargar los datos (asegúrate de tener el CSV en el mismo directorio o cambia la ruta)
df = pd.read_csv('student_habits_performance.csv')
df.head()


,student_id,age,gender,study_hours_per_day,social_media_hours,netflix_hours,part_time_job,attendance_percentage,sleep_hours,diet_quality,exercise_frequency,parental_education_level,internet_quality,mental_health_rating,extracurricular_participation,exam_score
0,S1000,23,Female,0.0,1.2,1.1,No,85.0,8.0,Fair,6,Master,Average,8,Yes,56.2
1,S1001,20,Female,6.9,2.8,2.3,No,97.3,4.6,Good,6,High School,Average,8,No,100.0
2,S1002,21,Male,1.4,3.1,1.3,No,94.8,8.0,Poor,1,High School,Poor,1,No,34.3
3,S1003,23,Female,1.0,3.9,1.0,No,71.0,9.2,Poor,4,Master,Good,1,Yes,26.8
4,S1004,19,Female,5.0,4.4,0.5,No,90.9,4.9,Fair,3,Master,Good,1,No,66.4


## Exploración y limpieza del dataset

In [3]:

# Información general del dataset
df.info()
df.describe()
df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   student_id                     1000 non-null   object 
 1   age                            1000 non-null   int64  
 2   gender                         1000 non-null   object 
 3   study_hours_per_day            1000 non-null   float64
 4   social_media_hours             1000 non-null   float64
 5   netflix_hours                  1000 non-null   float64
 6   part_time_job                  1000 non-null   object 
 7   attendance_percentage          1000 non-null   float64
 8   sleep_hours                    1000 non-null   float64
 9   diet_quality                   1000 non-null   object 
 10  exercise_frequency             1000 non-null   int64  
 11  parental_education_level       909 non-null    object 
 12  internet_quality               1000 non-null   ob

student_id                        0
age                               0
gender                            0
study_hours_per_day               0
social_media_hours                0
netflix_hours                     0
part_time_job                     0
attendance_percentage             0
sleep_hours                       0
diet_quality                      0
exercise_frequency                0
parental_education_level         91
internet_quality                  0
mental_health_rating              0
extracurricular_participation     0
exam_score                        0
dtype: int64

In [4]:

# Eliminar columna que no aporta información útil para el modelo
df = df.drop(columns=['student_id'])


## Definir variables predictoras (X) y variable objetivo (y)

In [5]:
# Suponiendo que 'performance' es la variable objetivo
X = df.drop(columns=['exam_score'])  
y = df['exam_score']

## Preprocesamiento: One Hot Encoding para variables categóricas

In [6]:
# 2. Detectar columnas categóricas
categorical_cols = X.select_dtypes(include='object').columns

# 3. Codificar variables categóricas
ohe = OneHotEncoder(drop='first', sparse_output=False)
X_encoded = pd.DataFrame(
    ohe.fit_transform(X[categorical_cols]),
    columns=ohe.get_feature_names_out(categorical_cols)
)

# 4. Concatenar columnas numéricas + codificadas
X_numeric = X.drop(columns=categorical_cols).reset_index(drop=True)
X_prepared = pd.concat([X_numeric, X_encoded], axis=1)

## Evaluación del modelo con validación cruzada (K-Fold)

In [7]:

# 4. Crear modelo de regresión
model = DecisionTreeRegressor(random_state=42)

# 5. Validación cruzada
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(model, X_prepared, y, cv=kfold, scoring='neg_mean_squared_error')

# 6. Mostrar resultados
import numpy as np
rmse_scores = np.sqrt(-scores)  # convertir MSE negativo a RMSE
print(f'RMSE promedio con 5-Fold CV: {rmse_scores.mean():.4f}')
print(f'Todas las puntuaciones RMSE: {rmse_scores}')

RMSE promedio con 5-Fold CV: 9.4689
Todas las puntuaciones RMSE: [ 9.54619558  9.31742185  8.7249298   9.61093388 10.14494455]


## Conclusión


El modelo de árbol de decisión ha sido entrenado y evaluado con validación cruzada.
Esto nos permite tener una mejor estimación de su rendimiento general en comparación con una sola partición de entrenamiento/test.
En próximos pasos se puede comparar con otros modelos como K-NN, SVM o redes neuronales.
